In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"

In [4]:
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)

In [7]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

In [10]:
#create examples
examples = [
    {
        "rev":"I love this product",
        "answer":"Positive"
    },
    {
        "rev":"It was an average experience",
        "answer":"neutral"
    },
    {
        "rev":"I wonder why this is so highly rated.",
        "answer":"negative"
    },
]

#create a temolate from the above examples
example_prompt = PromptTemplate(
                    input_variables=["rev","answer"],
                    template=""" 
                                User:{rev}
                                AI:{answer}
                                """
)

In [8]:
#Now break our previous prompt into a prefix and suffix
#the prefix is our instructions
prefix="Identify the sentiment of the user review. Here are some exaamples:"
#and the suffix our user input and output indicator
suffix="""
        User:{review}
        AI:
        """ 

In [11]:
#now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["review"],
    example_separator="\n"
)

In [12]:
user_review = "What is the meaning of life?"

In [14]:
message= few_shot_prompt_template.format(review=user_review)

In [16]:
print(message)

Identify the sentiment of the user review. Here are some exaamples:
 
                                User:I love this product
                                AI:Positive
                                
 
                                User:It was an average experience
                                AI:neutral
                                
 
                                User:I wonder why this is so highly rated.
                                AI:negative
                                

        User:What is the meaning of life?
        AI:
        


In [17]:
user_review_2 = "Well structured. Five stars."

In [18]:
message_2 = few_shot_prompt_template.format(review=user_review_2)

In [19]:
print(message_2)

Identify the sentiment of the user review. Here are some exaamples:
 
                                User:I love this product
                                AI:Positive
                                
 
                                User:It was an average experience
                                AI:neutral
                                
 
                                User:I wonder why this is so highly rated.
                                AI:negative
                                

        User:Well structured. Five stars.
        AI:
        


In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [21]:
chat = ChatGoogleGenerativeAI(model='gemini-1.5-pro-002')

In [24]:
chat_message_2 = chat.invoke(message_2)

In [25]:
chat_message_2

AIMessage(content='Positive', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-21c32354-5684-4c5e-8633-1b836f9a0d0a-0', usage_metadata={'input_tokens': 91, 'output_tokens': 2, 'total_tokens': 93, 'input_token_details': {'cache_read': 0}})

In [26]:
chat_message_1 = chat.invoke(message)

In [27]:
chat_message_1

AIMessage(content="Neutral.  The user isn't expressing an opinion on anything; they're asking a philosophical question.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-b0709c0d-e22a-4ffc-93e2-2c10af25a31f-0', usage_metadata={'input_tokens': 92, 'output_tokens': 23, 'total_tokens': 115, 'input_token_details': {'cache_read': 0}})